In [ ]:
import numpy as np
import random
import os

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import models, transforms, datasets

import matplotlib.pyplot as plt
import matplotlib.animation as animation

seed = 999
random.seed(seed)
torch.manual_seed(seed)

In [ ]:
img_size = 64
num_workers=1
dim_z = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ngpu = 1

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        
        self.structure = nn.Sequential(
            nn.ConvTranspose2d(in_channels=dim_z, out_channels=512, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )
        
    def forward(self, z):
        return self.structure(z)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        
        self.structure = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, img):
        return self.structure(img)

In [ ]:
data_dir = "./Human_Data"

In [ ]:
def weights_init(m):
    name = m.__class__.__name__
    if (name.find("Conv") != -1):
        nn.init.normal_(m.weight.data, 0., 0.02) # ~N(mean=0, std=0.02)
    elif (name.find("BatchNorm") != -1):
        nn.init.normal_(m.weight.data, 1., 0.02)
        nn.init.constant_(m.bias.data, 0.)

In [ ]:
model_G = Generator(ngpu=1)
if (device.type == "cuda" and ngpu > 1):
    model_G = nn.DataParallel(model_G, list(range(ngpu)))
# Weights initialization
model_G.apply(weights_init)

In [ ]:
model_D = Discriminator(ngpu=1)
if (device == "cuda" and ngpu > 1):
    model_D = nn.DataParallel(model_D, list(range(ngpu)))
# Initialize weights for the Discriminator
model_D.apply(weights_init)

In [ ]:
def create_dataloader(data_dir, img_size, bs):
    transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=bs, shuffle=True, num_workers=4)
    return dataloader
# Tạo hàm train mô hình DCGAN
def train_DCGAN(data_dir, bs, dim_z, NUM_EPOCHS, lr):
    # Khởi tạo DataLoader cho tập dữ liệu
    dataloader = create_dataloader(data_dir, img_size, bs)

    # Chọn hàm mất mát và tối ưu hóa
    criterion = nn.BCELoss()
    optimizerD = optim.Adam(model_D.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizerG = optim.Adam(model_G.parameters(), lr=lr, betas=(0.5, 0.999))
    model_D.to(device)
    model_G.to(device)

    # Lặp qua số epochs
    for epoch in range(NUM_EPOCHS):
        total_loss_D = 0.0
        total_loss_G = 0.0
        num_batches = len(dataloader)
        
        for images, _ in iter(dataloader):   
            model_D.zero_grad()
            # Define log(D(x))
            real_imgs = images.to(device)
            bs = real_imgs.size(0) # batch size
            
            labels = torch.full(size=(bs,), fill_value=1.0, dtype=torch.float, device=device)
            outputs = model_D(real_imgs).view(-1)     
            lossD_real = criterion(outputs, labels)
            lossD_real.backward() # this will be automatically accumulated with lossD_fake in PyTorch
            
            # Define log(1-D(G(z)))
            z = torch.randn(size=(bs, dim_z, 1, 1), device=device, requires_grad=False)
            fake_imgs = model_G(z)
            
            labels.fill_(0.0) # fake images
            outputs = model_D(fake_imgs.detach()).view(-1) 
            lossD_fake = criterion(outputs, labels)
            lossD_fake.backward() # this is automatically accumulated with lossD_real in PyTorch
            
            optimizerD.step()
            total_loss_D += (lossD_real + lossD_fake).item()

            model_G.zero_grad()
            
            labels.fill_(1.0)
            outputs = model_D(fake_imgs).view(-1) # now the fake images are given label "real" to optimize G
            lossG = criterion(outputs, labels)
            lossG.backward()
            
            optimizerG.step()
            total_loss_G += lossG.item()
        # In loss sau mỗi epoch
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] - Loss_D: {total_loss_D / num_batches:.4f}, Loss_G: {total_loss_G / num_batches:.4f}")
    mse_loss = torch.mean((real_imgs - fake_imgs) ** 2)  # Tính mean squared error
    return mse_loss.item()  # Trả về giá trị mean squared error

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
param_grid = {
    "NUM_EPOCHS": [30, 50, 100],
    "bs": [64, 128],
    "lr": [0.0001, 0.0002, 0.0005]
}

# Lặp qua các giá trị siêu tham số
best_loss = float("inf")
best_hyperparameters = None

for params in ParameterGrid(param_grid):
    NUM_EPOCHS = params["NUM_EPOCHS"]
    bs = params["bs"]
    lr = params["lr"]

    # Train mô hình với các siêu tham số hiện tại
    current_loss = train_DCGAN(data_dir, bs, dim_z, NUM_EPOCHS, lr)

    # Lưu siêu tham số tốt nhất nếu loss tốt hơn
    if current_loss < best_loss:
        best_loss = current_loss
        best_hyperparameters = params
        # Lưu model tốt nhất
        model_D_path = "./Model/best_model_D.pth"
        model_G_path = "./Model/best_model_G.pth"
        torch.save(model_D.state_dict(), model_D_path)
        torch.save(model_G.state_dict(), model_G_path)
# In ra các siêu tham số tối ưu
print("Best hyperparameters:", best_hyperparameters)